In [209]:
# inpired by https://github.com/aerdem4/kaggle-quora-dup
# useful: https://habrahabr.ru/company/dca/blog/274027/
# link to the contest: https://contest.yandex.ru/algorithm2018/contest/7914
import re
import numpy as np
import pandas as pd
import seaborn as sns
import gensim
import csv

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers.core import Lambda
from keras.layers.merge import concatenate, add, multiply
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.noise import GaussianNoise
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [321]:
train_columns = "context_id,context_2,context_1,context_0,reply_id,reply,label,confidence".split(',')
train = pd.read_csv("/home/andrei/yandex_ml_track/train.tsv", delimiter='\t', names=train_columns, quoting=csv.QUOTE_NONE)
train = train.fillna('')

test_columns = "context_id,context_2,context_1,context_0,reply_id,reply".split(',')
test = pd.read_csv("/home/andrei/yandex_ml_track/final.tsv", delimiter='\t', names=test_columns, quoting=csv.QUOTE_NONE)
test = test.fillna('')

res = {'good': 1, 'neutral': 0.1, 'bad': 0}
y = train['label'].map(res)

fx = lambda x: x >= 0.85

y = y*(train['confidence'].apply(fx))  # very important
train.drop('label', axis=1)

text_columns = ['context_2', 'context_1', 'context_0', 'reply']
train['text'] = test['text'] = ''
for c in text_columns[:-1]:
    train['text'] += train[c].astype('U')
    test['text'] += test[c].astype('U')

CPU times: user 753 ms, sys: 24.8 ms, total: 778 ms
Wall time: 778 ms


In [24]:
def write_to_submission_file(result, out_file):
    predicted_df = pd.DataFrame(result[['context_id', 'reply_id']],
                                columns=['context_id', 'reply_id'])
    predicted_df.to_csv(out_file, index=False, header=False, sep='\t')

In [5]:
# w2v related stuff
PATH_W2VEC = '/home/andrei/Downloads/all.norm-sz100-w10-cb0-it1-min100.w2v'
w2v = gensim.models.KeyedVectors.load_word2vec_format(PATH_W2VEC, binary=True, unicode_errors='ignore')

CPU times: user 20 s, sys: 887 ms, total: 20.8 s
Wall time: 21.2 s


In [7]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
_w2v = dict(zip(w2v.index2word, w2v.vectors))

In [324]:
vectorizer = mean_vectorizer(_w2v)

np.random.seed(0)
WNL = SnowballStemmer("russian")
STOP_WORDS = set(stopwords.words('russian'))
MAX_SEQUENCE_LENGTH = 35
MIN_WORD_OCCURRENCE = 100
REPLACE_WORD = "ПУСТОТА"
EMBEDDING_DIM = 100
NUM_FOLDS = 10
BATCH_SIZE = 1025


def words_cutter(word):
    if len(word) < 4:
        return word
    return WNL.stem(word)


def preprocess(s):
    for c in '[“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@]':
        s = s.replace(c, ' {} '.format(c))
    
    s = ' '.join(s.split())
    
    return s


def is_numeric(s):
    return any(i.isdigit() for i in s)


def prepare(q):
    new_q = []
    surplus_q = []
    numbers_q = []
    new_memento = True
    for w in q.split()[::-1]:
        if w in top_words:
            new_q = [w] + new_q
            new_memento = True
        elif w not in STOP_WORDS:
            if new_memento:
                new_q = [REPLACE_WORD] + new_q
                new_memento = False
            if is_numeric(w):
                numbers_q = [w] + numbers_q
            else:
                surplus_q = [w] + surplus_q
        else:
            new_memento = True
        if len(new_q) == MAX_SEQUENCE_LENGTH:
            break
    new_q = " ".join(new_q)
    return new_q, set(surplus_q), set(numbers_q)

def get_special_symbols_count(text):
    res = 0
    for c in '“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@':
        res += text.count(c)
    return res

def get_first_word(text):
    text = re.sub('[“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@]', ' ', text).split()
    return text[0] if text else ''

def get_last_word(text):
    text = re.sub('[“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@]', ' ', text)
    return text[-1] if text else ''

def get_word_count(text):
    text = re.sub('[“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@]', ' ', text)
    return len(text.split())

def get_features(dataset):
    cnt0 = dataset['text'].apply(get_special_symbols_count).reshape(len(dataset), 1)
    cnt1 = dataset['reply'].apply(get_special_symbols_count).reshape(len(dataset), 1)
    
    dataset['first_text_word'] = dataset['text'].apply(get_first_word)
    dataset['first_reply_word'] = dataset['reply'].apply(get_first_word)
    
    dataset['last_text_word'] = dataset['text'].apply(get_last_word)
    dataset['last_reply_word'] = dataset['reply'].apply(get_last_word)
    
    reply_columns = vectorizer.transform(dataset['text'])
    text_columns = vectorizer.transform(dataset['reply'])

    length0 = dataset['text'].apply(len).reshape(len(dataset), 1)
    length1 = dataset['reply'].apply(len).reshape(len(dataset), 1)
    words0 = dataset['text'].apply(get_word_count).reshape(len(dataset), 1)
    words1 = dataset['text'].apply(get_word_count).reshape(len(dataset), 1)
    
    first_word_text = vectorizer.transform(dataset['first_text_word'])
    last_word_text = vectorizer.transform(dataset['last_text_word'])
    
    first_word_reply = vectorizer.transform(dataset['first_reply_word'])
    last_word_reply = vectorizer.transform(dataset['last_reply_word'])
    
    print(length0.shape, reply_columns.shape)
    df = np.hstack([text_columns, reply_columns, length0, length1, cnt0, cnt1,
                    #first_word_text, first_word_reply, 
                    last_word_text, last_word_reply]) #, words0, words1])
    print('Shape: ', df.shape)
    return df

def extract_features(df):
    q1s = np.array([""] * len(df), dtype=object)
    q2s = np.array([""] * len(df), dtype=object)
    features = np.zeros((len(df), 4))

    for i, (q1, q2) in enumerate(list(zip(df["text"], df["reply"]))):
        q1s[i], surplus1, numbers1 = prepare(q1)
        q2s[i], surplus2, numbers2 = prepare(q2)
        features[i, 0] = len(surplus1.intersection(surplus2))
        features[i, 1] = len(surplus1.union(surplus2))
        features[i, 2] = len(numbers1.intersection(numbers2))
        features[i, 3] = len(numbers1.union(numbers2))
    
        
    w2v_mean_vec = get_features(df)
    
    return q1s, q2s, np.hstack([features, w2v_mean_vec])

SyntaxError: invalid syntax (<ipython-input-324-8cc1e9323484>, line 50)

In [325]:
t_train, r_train, f = extract_features(train)

tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(np.append(t_train, r_train))
data_1 = pad_sequences(tokenizer.texts_to_sequences(t_train), maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(tokenizer.texts_to_sequences(r_train), maxlen=MAX_SEQUENCE_LENGTH)

x1_tr, x1_val, y_tr, y_val = split(data_1, y, 0.7)
x2_tr, x2_val, y_tr, y_val = split(data_2, y, 0.7)
f_tr, f_val, y_tr, y_val = split(f, y, 0.7)
print(x1_tr.shape, x1_val.shape, x2_tr.shape, x2_val.shape, y_tr.shape, y_val.shape, y_tr.mean(), y_tr.mean())
print(f.shape)

word_index = tokenizer.word_index

nb_words = len(word_index) + 1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = _w2v.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
top_words = _w2v.keys()

/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:36: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:38: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packag

(97533, 1) (97533, 100)
Shape:  (97533, 204)
(68273, 35) (29260, 35) (68273, 35) (29260, 35) (68273,) (29260,) 0.31027931978966794 0.31027931978966794
(97533, 208)


In [326]:
#Model 1
# model = Sequential()
# model.add(Dense(128, input_dim=(Xtr.shape[1])))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('sigmoid'))
# model.compile(loss='binary_crossentropy',
#               optimizer='nadam',
#               metrics=['mse'])


#Model 2
# model = Sequential()
# model.add(Dense(128, input_dim=(Xtr.shape[1])))
# #model.add(BatchNormalization(batch_input_shape=(Xtr.shape[1],)))
# model.add(GaussianNoise(0.1))
# model.add(Dense(150, activation="relu"))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='nadam',
#               metrics=['mse'])


# history = model.fit(Xtr, ytr, 
#                     batch_size=128,
#                     epochs=50,
#                     validation_data=(Xval, yval),
#                     class_weight='auto',
#                     verbose=1)

embedding_layer = Embedding(nb_words,
          EMBEDDING_DIM,
          weights=[embedding_matrix],
          input_length=MAX_SEQUENCE_LENGTH,
          trainable=False)

lstm_layer = LSTM(75, recurrent_dropout=0.2)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

addition = add([x1, y1])
minus_y1 = Lambda(lambda x: -x)(y1)
merged = add([x1, minus_y1])
merged = multiply([merged, merged])
merged = concatenate([merged, addition])


features_input = Input(shape=(f.shape[1],), dtype="float32")
features_dense = BatchNormalization()(features_input)
features_dense = Dense(200, activation="relu")(features_dense)
features_dense = Dropout(0.2)(features_dense)

merged = concatenate([merged, features_dense])

merged = BatchNormalization()(merged)
merged = GaussianNoise(0.1)(merged)
merged = Dense(150, activation="relu")(merged)
merged = Dropout(0.2)(merged)
merged = BatchNormalization()(merged)
merged = Dense(1)(merged)
merged = Activation('sigmoid')(merged)

out = Dropout(0.4)(merged)

model = Model(inputs=[sequence_1_input, sequence_2_input, features_input]
              , outputs=out)

model.compile(loss='binary_crossentropy', #'mean_squared_error'
              optimizer='nadam',
              metrics=['mse'])

In [327]:
best_model_path = "/home/andrei/yandex_ml_track/best_model_weights.h5" 
model_checkpoint = ModelCheckpoint(best_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([x1_tr, x2_tr, f_tr], y_tr, validation_data=([x1_val, x2_val, f_val], y_val), 
                 epochs=300, batch_size=1025, shuffle=True, callbacks=[model_checkpoint,], verbose=1)

model.load_weights(best_model_path)

Train on 68273 samples, validate on 29260 samples
Epoch 1/300
68273/68273 [==============================] - 57s 836us/step - loss: 3.6642 - mean_squared_error: 0.4188 - val_loss: 0.8067 - val_mean_squared_error: 0.2410
Epoch 2/300
68273/68273 [==============================] - 52s 767us/step - loss: 2.4938 - mean_squared_error: 0.2703 - val_loss: 0.6703 - val_mean_squared_error: 0.2171
Epoch 3/300
68273/68273 [==============================] - 53s 771us/step - loss: 2.4059 - mean_squared_error: 0.2536 - val_loss: 0.6977 - val_mean_squared_error: 0.2222
Epoch 4/300
68273/68273 [==============================] - 52s 765us/step - loss: 2.4120 - mean_squared_error: 0.2521 - val_loss: 0.6522 - val_mean_squared_error: 0.2173
Epoch 5/300
68273/68273 [==============================] - 52s 766us/step - loss: 2.3858 - mean_squared_error: 0.2484 - val_loss: 0.6466 - val_mean_squared_error: 0.2172
Epoch 6/300
68273/68273 [==============================] - 54s 793us/step - loss: 2.3880 - mean_squa

Epoch 49/300
68273/68273 [==============================] - 52s 768us/step - loss: 2.3535 - mean_squared_error: 0.2349 - val_loss: 0.6299 - val_mean_squared_error: 0.2095
Epoch 50/300
68273/68273 [==============================] - 52s 767us/step - loss: 2.3459 - mean_squared_error: 0.2319 - val_loss: 0.6360 - val_mean_squared_error: 0.2116
Epoch 51/300
68273/68273 [==============================] - 52s 768us/step - loss: 2.3449 - mean_squared_error: 0.2330 - val_loss: 0.6506 - val_mean_squared_error: 0.2160
Epoch 52/300
68273/68273 [==============================] - 53s 773us/step - loss: 2.3652 - mean_squared_error: 0.2334 - val_loss: 0.6862 - val_mean_squared_error: 0.2244
Epoch 53/300
68273/68273 [==============================] - 53s 770us/step - loss: 2.3004 - mean_squared_error: 0.2284 - val_loss: 0.6521 - val_mean_squared_error: 0.2165
Epoch 54/300
68273/68273 [==============================] - 53s 780us/step - loss: 2.3270 - mean_squared_error: 0.2305 - val_loss: 0.6657 - val_m

Epoch 97/300
68273/68273 [==============================] - 53s 771us/step - loss: 2.2986 - mean_squared_error: 0.2170 - val_loss: 0.6295 - val_mean_squared_error: 0.2117
Epoch 98/300
68273/68273 [==============================] - 53s 772us/step - loss: 2.2992 - mean_squared_error: 0.2215 - val_loss: 0.6943 - val_mean_squared_error: 0.2191
Epoch 99/300
68273/68273 [==============================] - 53s 773us/step - loss: 2.2971 - mean_squared_error: 0.2170 - val_loss: 0.7072 - val_mean_squared_error: 0.2214
Epoch 100/300
68273/68273 [==============================] - 53s 772us/step - loss: 2.2759 - mean_squared_error: 0.2137 - val_loss: 0.7276 - val_mean_squared_error: 0.2238
Epoch 101/300
68273/68273 [==============================] - 53s 772us/step - loss: 2.2751 - mean_squared_error: 0.2157 - val_loss: 0.6297 - val_mean_squared_error: 0.2114
Epoch 102/300
68273/68273 [==============================] - 53s 772us/step - loss: 2.3647 - mean_squared_error: 0.2400 - val_loss: 0.6060 - va

68273/68273 [==============================] - 53s 781us/step - loss: 2.3001 - mean_squared_error: 0.2053 - val_loss: 0.7369 - val_mean_squared_error: 0.2200
Epoch 145/300
68273/68273 [==============================] - 53s 778us/step - loss: 2.2382 - mean_squared_error: 0.2009 - val_loss: 0.7396 - val_mean_squared_error: 0.2202
Epoch 146/300
68273/68273 [==============================] - 53s 778us/step - loss: 2.2310 - mean_squared_error: 0.2009 - val_loss: 0.7749 - val_mean_squared_error: 0.2231
Epoch 147/300
68273/68273 [==============================] - 53s 776us/step - loss: 2.2754 - mean_squared_error: 0.2038 - val_loss: 0.7637 - val_mean_squared_error: 0.2233
Epoch 148/300
68273/68273 [==============================] - 53s 778us/step - loss: 2.2629 - mean_squared_error: 0.2005 - val_loss: 0.7512 - val_mean_squared_error: 0.2212
Epoch 149/300
68273/68273 [==============================] - 53s 779us/step - loss: 2.2491 - mean_squared_error: 0.1991 - val_loss: 0.7613 - val_mean_squa

Epoch 192/300
68273/68273 [==============================] - 53s 772us/step - loss: 2.2669 - mean_squared_error: 0.2007 - val_loss: 0.7857 - val_mean_squared_error: 0.2230
Epoch 193/300
68273/68273 [==============================] - 53s 774us/step - loss: 2.2333 - mean_squared_error: 0.1961 - val_loss: 0.8399 - val_mean_squared_error: 0.2281
Epoch 194/300
68273/68273 [==============================] - 53s 776us/step - loss: 2.2584 - mean_squared_error: 0.1993 - val_loss: 0.7870 - val_mean_squared_error: 0.2236
Epoch 195/300
68273/68273 [==============================] - 53s 776us/step - loss: 2.2141 - mean_squared_error: 0.1940 - val_loss: 0.7874 - val_mean_squared_error: 0.2220
Epoch 196/300
68273/68273 [==============================] - 53s 774us/step - loss: 2.2576 - mean_squared_error: 0.1937 - val_loss: 0.7725 - val_mean_squared_error: 0.2223
Epoch 197/300
68273/68273 [==============================] - 53s 774us/step - loss: 2.2334 - mean_squared_error: 0.1935 - val_loss: 0.8153 -

68273/68273 [==============================] - 53s 774us/step - loss: 2.2378 - mean_squared_error: 0.1937 - val_loss: 0.7129 - val_mean_squared_error: 0.2171
Epoch 240/300
68273/68273 [==============================] - 53s 778us/step - loss: 2.2605 - mean_squared_error: 0.2003 - val_loss: 0.8500 - val_mean_squared_error: 0.2386
Epoch 241/300
68273/68273 [==============================] - 53s 774us/step - loss: 2.3154 - mean_squared_error: 0.2212 - val_loss: 0.7801 - val_mean_squared_error: 0.2286
Epoch 242/300
68273/68273 [==============================] - 53s 775us/step - loss: 2.2828 - mean_squared_error: 0.2187 - val_loss: 0.7259 - val_mean_squared_error: 0.2202
Epoch 243/300
68273/68273 [==============================] - 53s 772us/step - loss: 2.2886 - mean_squared_error: 0.2098 - val_loss: 0.7828 - val_mean_squared_error: 0.2267
Epoch 244/300
68273/68273 [==============================] - 53s 777us/step - loss: 2.2554 - mean_squared_error: 0.2028 - val_loss: 0.8276 - val_mean_squa

Epoch 287/300
68273/68273 [==============================] - 53s 777us/step - loss: 2.1738 - mean_squared_error: 0.1821 - val_loss: 0.8905 - val_mean_squared_error: 0.2319
Epoch 288/300
68273/68273 [==============================] - 53s 775us/step - loss: 2.2020 - mean_squared_error: 0.1845 - val_loss: 0.8663 - val_mean_squared_error: 0.2302
Epoch 289/300
68273/68273 [==============================] - 53s 777us/step - loss: 2.1977 - mean_squared_error: 0.1857 - val_loss: 0.8066 - val_mean_squared_error: 0.2251
Epoch 290/300
68273/68273 [==============================] - 53s 779us/step - loss: 2.3091 - mean_squared_error: 0.2134 - val_loss: 0.7328 - val_mean_squared_error: 0.2274
Epoch 291/300
68273/68273 [==============================] - 53s 778us/step - loss: 2.3359 - mean_squared_error: 0.2200 - val_loss: 0.6705 - val_mean_squared_error: 0.2131
Epoch 292/300
68273/68273 [==============================] - 53s 779us/step - loss: 2.3043 - mean_squared_error: 0.2136 - val_loss: 0.7193 -

In [328]:
t_test, r_test, f = extract_features(test)
test_1 = pad_sequences(tokenizer.texts_to_sequences(t_test), maxlen=MAX_SEQUENCE_LENGTH)
test_2 = pad_sequences(tokenizer.texts_to_sequences(r_test), maxlen=MAX_SEQUENCE_LENGTH)

result = model.predict([test_1, test_2, f], batch_size=1025, verbose=1)

/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:36: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packages/ipykernel_launcher.py:38: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/andrei/.local/lib/python3.5/site-packag

(104834, 1) (104834, 100)
Shape:  (104834, 204)
104834/104834 [==============================] - 32s 303us/step


In [329]:
test_for_predict = test.copy()
test_for_predict['predicted_labels'] = result
test_for_predict = test_for_predict.reindex(test_for_predict.sort_values(['context_id', 'predicted_labels'],
                                                                         ascending=[False, False]).index)
print(test_for_predict[['context_id', 'predicted_labels', 'reply_id']])
write_to_submission_file(test_for_predict, out_file='/home/andrei/yandex_ml_track/test_yandex1.csv')

             context_id  predicted_labels  reply_id
104829  281440204584682          0.464357         1
104830  281440204584682          0.416324         2
104833  281440204584682          0.361383         5
104828  281440204584682          0.333109         0
104831  281440204584682          0.325704         3
104832  281440204584682          0.170056         4
104822  281438845749348          0.436052         0
104825  281438845749348          0.422779         3
104824  281438845749348          0.417736         2
104827  281438845749348          0.412484         5
104826  281438845749348          0.410793         4
104823  281438845749348          0.196164         1
104818  281434882461340          0.463625         2
104817  281434882461340          0.389298         1
104816  281434882461340          0.359067         0
104820  281434882461340          0.352215         4
104821  281434882461340          0.333618         5
104819  281434882461340          0.265449         3
104811  2813